In [64]:
from classification import StockPredictor
from DataLoader import DataLoader
import pandas as pd

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
ticker = 'AAPL'
dataLoader = DataLoader() # if yahoo does not work use "dataLoader.get_data_locally('AAPL')"
stock_data = dataLoader.pipeline(ticker)

In [51]:
stock_data

,open,close,high,low,volume,max_5,min_5,max_10,min_10,max_20,...,stochastic_fast,stochastic_slow,%r,atr,cmo,cci,mom,bias,wnr,target
0,0.064816,0.064816,0.065245,0.064816,34272000,0.065674,0.063528,0.065674,0.063099,0.066962,...,69.987738,73.320430,42.871654,0.001196,33.333058,38.811032,0.003005,0.666648,-0.333335,0
1,0.064816,0.064816,0.065245,0.064816,37408000,0.065674,0.064386,0.065674,0.063528,0.066962,...,69.987738,66.655047,50.014161,0.001196,22.221858,38.747777,0.001717,0.398933,-0.388891,0
2,0.064816,0.064386,0.065245,0.064386,76092800,0.065674,0.064386,0.065674,0.063528,0.065674,...,59.989664,66.655047,66.675529,0.001196,-6.666482,31.136203,-0.000429,-0.199563,-0.533332,0
3,0.063099,0.062240,0.063099,0.062240,25244800,0.064816,0.06224,0.065674,0.06224,0.065674,...,9.998248,46.658550,100.000000,0.001134,-22.221708,-88.316523,-0.001717,-3.268670,-0.611109,1
4,0.063957,0.063957,0.064386,0.063957,31315200,0.064816,0.06224,0.065674,0.06224,0.065674,...,44.435023,38.140978,55.564977,0.001257,0.000000,-7.833457,0.000000,-0.600405,-0.500000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10937,206.089996,205.350006,206.990005,202.160004,101010600,213.320007,205.350006,213.320007,193.160004,213.320007,...,62.787913,84.968448,37.212087,6.983572,26.197218,28.654023,8.370010,-0.685311,-0.369014,0
10938,203.100006,198.889999,204.100006,198.210007,69018500,213.320007,198.889999,213.320007,198.889999,213.320007,...,36.686876,65.041315,84.292637,6.437143,16.565463,3.639740,5.729996,-4.075433,-0.417173,0
10939,198.210007,198.509995,200.649994,197.020004,51216500,213.320007,198.509995,213.320007,198.509995,213.320007,...,35.151503,44.875431,91.505184,6.431428,-4.332548,-11.938346,-1.230011,-4.201879,-0.521663,0
10940,199.169998,196.250000,199.440002,193.250000,68536700,213.320007,196.25,213.320007,196.25,213.320007,...,26.020212,32.619530,85.922101,6.175714,-32.376903,-40.056531,-8.350006,-4.909343,-0.661885,1


In [72]:
stock_predictor = StockPredictor(stock_data=stock_data, 
                                 model_names_list=["lgbm", "mlp", "xgboost", "randomforest"],
                                 ensemble_strategy="majority_vote",
                                 drift_name='adwin',
                                 feature_selector_name='selectkbest_sklearn',
                                 learning_threshold = 1000,
                                 ensemble_params={'n_models': 10}
                                 )
stock_predictor.prediction()

           Precision   Recall   F1       Support  
                                                  
       0      50.34%   69.83%   58.51%      4916  
       1      52.51%   32.63%   40.25%      5026  
                                                  
   Macro      51.43%   51.23%   49.38%            
   Micro      51.03%   51.03%   51.03%            
Weighted      51.44%   51.03%   49.28%            

                 51.03% accuracy                  


(0.51,
    class  precision  recall     f1
 0      0      0.503   0.698  0.585
 1      1      0.525   0.326  0.403)

In [71]:
stock_predictor.drifts_detected

0

In [ ]:
result_rows = []
for base_model in ["hoeffdingtreeclassifier", "extremelyfastdecisiontreeclassifier"]:
    for ensemble in ['BaggingClassifier', 'AdaBoostClassifier', 'ADWINBaggingClassifier', 'SRPClassifier', 'LeveragingBaggingClassifier']:
        stock_predictor = StockPredictor(stock_data=stock_data, 
                                 model_names_list=[base_model],
                                 ensemble_strategy=ensemble,
                                 drift_name='adwin',
                                 feature_selector_name='selectkbest',
                                 learning_threshold = 1000,
                                 ensemble_params={'n_models': 5}
                                 )
        accuracy, metrics_result = stock_predictor.prediction()
        print(base_model, ensemble, accuracy)
        result_rows.append([base_model, ensemble, 'adwin', 'selectkbest', 1000, 1, round(accuracy, 3), stock_predictor.drifts_detected, {}, ticker])
        
result_df = pd.DataFrame(result_rows, columns=['ensemble_name', 'model_name', 'drift_name', 'feature_selector_name', 'learning_threshold', 'iteration', 'accuracy', 'drifts_detected', 'model_args', 'ticker'])
result_df.to_csv('ensemble_results_df.csv', index=False)

           Precision   Recall   F1       Support  
                                                  
       0      51.20%   55.21%   53.13%      4916  
       1      52.55%   48.53%   50.46%      5026  
                                                  
   Macro      51.88%   51.87%   51.79%            
   Micro      51.83%   51.83%   51.83%            
Weighted      51.88%   51.83%   51.78%            

                 51.83% accuracy                  
hoeffdingtreeclassifier BaggingClassifier 0.518
           Precision   Recall   F1       Support  
                                                  
       0      48.77%   30.78%   37.74%      4916  
       1      50.25%   68.38%   57.93%      5026  
                                                  
   Macro      49.51%   49.58%   47.84%            
   Micro      49.79%   49.79%   49.79%            
Weighted      49.52%   49.79%   47.95%            

                 49.79% accuracy                  
hoeffdingtreeclassifier AdaBoost

In [ ]:
result_rows = []
for base_model in ["hoeffdingtreeclassifier", "extremelyfastdecisiontreeclassifier"]:
    for ensemble in ['BaggingClassifier', 'AdaBoostClassifier', 'ADWINBaggingClassifier', 'SRPClassifier', 'LeveragingBaggingClassifier']:
        stock_predictor = StockPredictor(stock_data=stock_data, 
                                 model_names_list=[base_model],
                                 ensemble_strategy=ensemble,
                                 drift_name='adwin',
                                 feature_selector_name='selectkbest',
                                 learning_threshold = 1000,
                                 ensemble_params={'n_models': 10}
                                 )
        accuracy, metrics_result = stock_predictor.prediction()
        print(base_model, ensemble, accuracy)
        result_rows.append([base_model, ensemble, 'adwin', 'selectkbest', 1000, 1, round(accuracy, 3), stock_predictor.drifts_detected, {}, ticker])
        
result_df = pd.DataFrame(result_rows, columns=['ensemble_name', 'model_name', 'drift_name', 'feature_selector_name', 'learning_threshold', 'iteration', 'accuracy', 'drifts_detected', 'model_args', 'ticker'])
result_df.to_csv('ensemble_results_df_2.csv', index=False)

           Precision   Recall   F1       Support  
                                                  
       0      51.52%   41.15%   45.75%      4916  
       1      51.90%   62.12%   56.55%      5026  
                                                  
   Macro      51.71%   51.63%   51.15%            
   Micro      51.75%   51.75%   51.75%            
Weighted      51.71%   51.75%   51.21%            

                 51.75% accuracy                  
hoeffdingtreeclassifier BaggingClassifier 0.518
           Precision   Recall   F1       Support  
                                                  
       0      49.66%   45.14%   47.29%      4916  
       1      50.73%   55.25%   52.90%      5026  
                                                  
   Macro      50.20%   50.20%   50.09%            
   Micro      50.25%   50.25%   50.25%            
Weighted      50.20%   50.25%   50.13%            

                 50.25% accuracy                  
hoeffdingtreeclassifier AdaBoost